## 1. Importações e Configuração

In [ ]:
import sys
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from loguru import logger

# Adicionar diretório de scripts ao path
project_root = Path.cwd().parent
scripts_path = project_root / "scripts"
sys.path.insert(0, str(scripts_path))

# Importar adapter síncrono NWS Stations
from api.services.nws_stations.nws_stations_sync_adapter import NWSStationsSyncAdapter

# Configurar logging
logger.remove()
logger.add(sys.stdout, level="INFO")

# Configurar estilo dos gráficos
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Importações concluídas")

## 2. Inicializar Cliente NWS Stations

In [ ]:
# Criar adapter síncrono (sem cache para demo)
nws_adapter = NWSStationsSyncAdapter()

# Verificar informações da API
info = nws_adapter.get_info()
print("\n📊 NWS Stations API Information:")
for key, value in info.items():
    print(f"  {key}: {value}")

## 3. Baixar Dados Observacionais - Chicago, IL

Vamos baixar dados observacionais de estações meteorológicas próximas a **Chicago, IL**.

**Coordenadas**: 41.8781°N, -87.6298°W  
**Período**: Últimos 7 dias

In [ ]:
# Definir localização: Chicago, IL
lat = 41.8781
lon = -87.6298
location_name = "Chicago, IL"

# Definir período: últimos 7 dias
end_date = datetime.now()
start_date = end_date - timedelta(days=7)

print(f"📍 Localização: {location_name}")
print(f"📅 Período: {start_date.date()} até {end_date.date()}")
print(f"🌐 Coordenadas: {lat:.4f}°, {lon:.4f}°")

In [ ]:
# Baixar dados observacionais
print("\n🌐 Baixando dados observacionais do NWS Stations...\n")
print("🔍 Buscando estações próximas e coletando observações...\n")

chicago_data = nws_adapter.get_daily_data_sync(
    lat=lat,
    lon=lon,
    start_date=start_date,
    end_date=end_date
)

print(f"\n✅ Dados baixados: {len(chicago_data)} registros diários")

## 4. Converter para DataFrame e Explorar

In [ ]:
# Converter para DataFrame
df = pd.DataFrame([
    {
        'date': record.date,
        'temp_max': record.temp_max,
        'temp_min': record.temp_min,
        'temp_mean': record.temp_mean,
        'humidity': record.humidity,
        'wind_speed': record.wind_speed,
        'solar_radiation': record.solar_radiation,
        'precipitation': record.precipitation
    }
    for record in chicago_data
])

# Exibir primeiras linhas
print("\n📋 Dados Observacionais (agregados diários):")
display(df.head())

# Verificar completude dos dados
print("\n📊 Completude dos Dados:")
completeness = df.notna().sum() / len(df) * 100
for col, pct in completeness.items():
    if col != 'date':
        print(f"  {col}: {pct:.1f}%")

In [ ]:
# Estatísticas descritivas
print("\n📊 Estatísticas Descritivas:")
display(df.describe())

## 5. Visualizações

### 5.1 Temperaturas Observadas

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df['date'], df['temp_max'], label='Temp Máxima', color='red', linewidth=2, marker='o')
ax.plot(df['date'], df['temp_mean'], label='Temp Média', color='orange', linewidth=2, marker='s')
ax.plot(df['date'], df['temp_min'], label='Temp Mínima', color='blue', linewidth=2, marker='v')

ax.fill_between(df['date'], df['temp_min'], df['temp_max'], alpha=0.2, color='gray')

ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Temperatura (°C)', fontsize=12)
ax.set_title(f'Temperaturas Observadas - {location_name}\nFonte: NWS Stations (NOAA)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 5.2 Precipitação e Umidade Observadas

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Precipitação
ax1.bar(df['date'], df['precipitation'], color='steelblue', alpha=0.7)
ax1.set_ylabel('Precipitação (mm)', fontsize=12)
ax1.set_title(f'Precipitação Observada - {location_name}', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Umidade Relativa
ax2.plot(df['date'], df['humidity'], color='green', linewidth=2, marker='o')
ax2.fill_between(df['date'], df['humidity'], alpha=0.3, color='green')
ax2.set_xlabel('Data', fontsize=12)
ax2.set_ylabel('Umidade Relativa (%)', fontsize=12)
ax2.set_title('Umidade Relativa Média Observada', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
plt.tight_layout()
plt.show()

### 5.3 Velocidade do Vento Observada

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df['date'], df['wind_speed'], color='purple', linewidth=2, marker='s')
ax.fill_between(df['date'], df['wind_speed'], alpha=0.3, color='purple')

ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Velocidade do Vento (m/s)', fontsize=12)
ax.set_title(f'Velocidade do Vento Média Observada - {location_name}', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 6. Exemplo 2: Miami, FL

In [ ]:
# Definir localização: Miami, FL
lat_miami = 25.7617
lon_miami = -80.1918
location_miami = "Miami, FL"

print(f"📍 Localização: {location_miami}")
print(f"📅 Período: {start_date.date()} até {end_date.date()}")
print(f"🌐 Coordenadas: {lat_miami:.4f}°, {lon_miami:.4f}°")

# Baixar dados
print("\n🌐 Baixando dados observacionais do NWS Stations...\n")

miami_data = nws_adapter.get_daily_data_sync(
    lat=lat_miami,
    lon=lon_miami,
    start_date=start_date,
    end_date=end_date
)

print(f"\n✅ Dados baixados: {len(miami_data)} registros")

In [ ]:
# Converter para DataFrame
df_miami = pd.DataFrame([
    {
        'date': record.date,
        'temp_max': record.temp_max,
        'temp_min': record.temp_min,
        'temp_mean': record.temp_mean,
        'humidity': record.humidity,
        'wind_speed': record.wind_speed,
        'precipitation': record.precipitation
    }
    for record in miami_data
])

# Visualizar
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df_miami['date'], df_miami['temp_max'], label='Temp Máxima', color='red', linewidth=2, marker='o')
ax.plot(df_miami['date'], df_miami['temp_mean'], label='Temp Média', color='orange', linewidth=2, marker='s')
ax.plot(df_miami['date'], df_miami['temp_min'], label='Temp Mínima', color='blue', linewidth=2, marker='v')
ax.fill_between(df_miami['date'], df_miami['temp_min'], df_miami['temp_max'], alpha=0.2, color='gray')

ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Temperatura (°C)', fontsize=12)
ax.set_title(f'Temperaturas Observadas - {location_miami}\nFonte: NWS Stations (NOAA)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Comparar Chicago vs Miami

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Chicago
ax.plot(df['date'], df['temp_mean'], label=f'{location_name} (Temp Média)', 
        color='blue', linewidth=2, marker='o')

# Miami
ax.plot(df_miami['date'], df_miami['temp_mean'], label=f'{location_miami} (Temp Média)', 
        color='red', linewidth=2, marker='s')

ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Temperatura (°C)', fontsize=12)
ax.set_title('Comparação: Temperaturas Observadas\nChicago (Norte) vs Miami (Sul)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Estatísticas comparativas
print("\n📊 Estatísticas Comparativas (Temperatura Média):")
print(f"  {location_name}: {df['temp_mean'].mean():.2f}°C (±{df['temp_mean'].std():.2f}°C)")
print(f"  {location_miami}: {df_miami['temp_mean'].mean():.2f}°C (±{df_miami['temp_mean'].std():.2f}°C)")

## 8. Health Check

In [ ]:
# Verificar disponibilidade da API
is_healthy = nws_adapter.health_check_sync()

if is_healthy:
    print("✅ NWS Stations API está funcionando corretamente")
else:
    print("❌ NWS Stations API não está acessível")

## 9. Salvar Dados

In [ ]:
# Salvar dados de Chicago
output_chicago = project_root / "data" / "csv" / "nws_stations_chicago_demo.csv"
output_chicago.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_chicago, index=False)
print(f"\n💾 Dados de Chicago salvos em: {output_chicago}")

# Salvar dados de Miami
output_miami = project_root / "data" / "csv" / "nws_stations_miami_demo.csv"
df_miami.to_csv(output_miami, index=False)
print(f"💾 Dados de Miami salvos em: {output_miami}")

---

## 📚 Referências

**NWS Stations (National Weather Service - NOAA)**
- Website: https://www.weather.gov/
- API Documentation: https://www.weather.gov/documentation/services-web-api
- Citation: National Weather Service, National Oceanic and Atmospheric Administration (NOAA)

**Licença**: US Government Public Domain

**Rede de Estações**: ~1,800 estações meteorológicas NOAA nos EUA

**Known Issues**:
- Observações atrasadas (>20min MADIS delay) - filtradas opcionalmente
- Valores nulos em temperaturas (max/min fora CST) - skipados
- Precipitação <0.4" rounding down - mantida com warning

---